# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
from scipy import io, signal
from matplotlib import pyplot as plt
import matplotlib.mlab as mlab

fs      = 125  # All signals were sampled at 125 HZ
min_freq= 0.67 # Corresponding to 40 BPM
max_freq= 3.2  # Corresponding to ~200 BPM

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls  = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    data
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    # are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]
    
    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def PassFilter(signal, passband):
    b, a = sp.signal.butter(3, passband, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
   
    # Compute aggregate error metric
    errs  = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)


def RunPulseRateAlgorithm(data_fl, ref_fl): 
    """ Calculates mean absolute errors and confidence values
    
    Args: 
        data_fl: (str) filepath to a troika .mat file.
        ref_f1: (str) filepath to a troika .mat file which contains reference heart beat estimates.
        
    Returns:
        error_array: Array of absolute errors
        conf_array: Array of confidence values for each error value.
    """
    
    # Load data
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    ground_truth = sp.io.loadmat(ref_fl)['BPM0']    
    # Convert into a 1D array
    ground_truth = ground_truth.reshape(-1)
    
    # Band pass ppg and acc signals 
    bandpassed_ppg = PassFilter(ppg, (min_freq, max_freq))
    
    # Splits of 8-second windows with 6-second overlaps since ground truth slides every 2 seconds
    spec_ppg, freqs_ppg, _  = mlab.specgram(bandpassed_ppg, Fs=fs, NFFT=fs*8, noverlap = 6*fs, pad_to=12*fs)
    # plt.xlabel('Time')
    # plt.ylabel('Frequency')
    # plt.title("Bandpass filtered PPG signal")
    accx = PassFilter(accx, (min_freq, max_freq))
    accy = PassFilter(accy, (min_freq, max_freq))
    accz = PassFilter(accz, (min_freq, max_freq))

    bandpassed_acc_mag = np.sqrt(np.sum(np.square(np.vstack((accx, accy, accz))), axis=0))
    
    # Specgram of the band-passed time-domain signals
    spec_acc, freqs_acc, _    = mlab.specgram(bandpassed_acc_mag, Fs=fs, NFFT=fs*8, noverlap = 6*fs, pad_to=12*fs)
    spec_accx, freqs_accx, _  = mlab.specgram(accx, Fs=fs, NFFT=fs*8, noverlap = 6*fs, pad_to=12*fs)
    spec_accy, freqs_accy, _  = mlab.specgram(accy, Fs=fs, NFFT=fs*8, noverlap = 6*fs, pad_to=12*fs)
    spec_accz, freqs_accz, _  = mlab.specgram(accz, Fs=fs, NFFT=fs*8, noverlap = 6*fs, pad_to=12*fs)

    # Pass through a frequency band that makes physiological sense.
    # This reduces the number of frequencies
    freqs_ppg  = freqs_ppg [(freqs_ppg >= min_freq) & (freqs_ppg <= max_freq)]
    freqs_acc  = freqs_acc [(freqs_acc >= min_freq) & (freqs_acc <= max_freq)]
    freqs_accx = freqs_accx [(freqs_accx >= min_freq) & (freqs_accx <= max_freq)]
    freqs_accy = freqs_accy [(freqs_accy >= min_freq) & (freqs_accy <= max_freq)]
    freqs_accz = freqs_accz [(freqs_accz >= min_freq) & (freqs_accz <= max_freq)]

    # Corresponding to applying the band pass on the frequencies, we have to "sanitize" the spectrum array 
    # as well. So the number of rows (spec_ppg.shape[0]) will become shorter. The number of columns will remain the
    # same, corresponding to the number of time blocks
    spec_ppg  = spec_ppg [(freqs_ppg >= min_freq) & (freqs_ppg <= max_freq)]
    spec_acc  = spec_acc [(freqs_acc >= min_freq) & (freqs_acc <= max_freq)]
    spec_accx = spec_accx [(freqs_accx >= min_freq) & (freqs_accx <= max_freq)]
    spec_accy = spec_accy [(freqs_accy >= min_freq) & (freqs_accy <= max_freq)]
    spec_accz = spec_accz [(freqs_accz >= min_freq) & (freqs_accz <= max_freq)]
    
    # spec_ppg is a 2D array. The COLUMNS correspond to each time block. The rows within each column correspond to 
    # frequency values
    num_time_blocks = spec_ppg.shape[1]
    num_frequencies = freqs_ppg.shape[0]
    #print ("num=", num_time_blocks)
    
    # Sort in descending order, so -1*spec
    # Each row corresponds to one time block
    sorted_spectrum_ppg  = (-spec_ppg).argsort(axis=0)
    sorted_spectrum_acc  = (-spec_acc).argsort(axis=0)
    sorted_spectrum_accx = (-spec_accx).argsort(axis=0)
    sorted_spectrum_accy = (-spec_accy).argsort(axis=0)
    sorted_spectrum_accz = (-spec_accz).argsort(axis=0)

    predicted_freqs = []
    for i in range(num_time_blocks):
        flag = 0
        
        # Check if dominant frequency in PPG overlaps with the top-3 peaks in ACCX, ACCY, ACCZ
        # and ACC_MAGNITUDE
        if (freqs_ppg[sorted_spectrum_ppg[0][i]]   == freqs_accx[sorted_spectrum_accx[0][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_accy[sorted_spectrum_accy[0][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_accz[sorted_spectrum_accz[0][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_acc[sorted_spectrum_acc[0][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_accx[sorted_spectrum_accx[1][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_accy[sorted_spectrum_accy[1][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_accz[sorted_spectrum_accz[1][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_acc[sorted_spectrum_acc[1][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_accx[sorted_spectrum_accx[2][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_accy[sorted_spectrum_accy[2][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_accz[sorted_spectrum_accz[2][i]]):
            flag = 1
        elif (freqs_ppg[sorted_spectrum_ppg[0][i]] == freqs_acc[sorted_spectrum_acc[2][i]]):
            flag = 1
        else:
            #print ("Dominant Freq Chosen as Heart Rate")
            predicted_freqs.append (freqs_ppg[sorted_spectrum_ppg[0][i]])
        
        # Check if second dominant frequency in PPG overlaps with the top-3 peaks in ACCX, ACCY, ACCZ
        # and ACC_MAGNITUDE
        if (flag):         
            flag = 0
            if (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_accx[sorted_spectrum_accx[0][i]]):
                 flag = 1
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_accy[sorted_spectrum_accy[0][i]]):
                 flag = 1
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_accz[sorted_spectrum_accz[0][i]]):
                 flag = 1   
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_acc[sorted_spectrum_acc[0][i]]):
                 flag = 1   
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_accx[sorted_spectrum_accx[1][i]]):
                 flag = 1
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_accy[sorted_spectrum_accy[1][i]]):
                 flag = 1
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_accz[sorted_spectrum_accz[1][i]]):
                 flag = 1 
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_acc[sorted_spectrum_acc[1][i]]):
                 flag = 1 
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_accx[sorted_spectrum_accx[2][i]]):
                 flag = 1
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_accy[sorted_spectrum_accy[2][i]]):
                 flag = 1
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_accz[sorted_spectrum_accz[2][i]]):
                 flag = 1   
            elif (freqs_ppg[sorted_spectrum_ppg[1][i]] == freqs_acc[sorted_spectrum_acc[2][i]]):
                 flag = 1 
                
            if (flag):
                #print ("Second Dominant Freq Chosen as Heart Rate")
                predicted_freqs.append (freqs_ppg[sorted_spectrum_ppg[2][i]]) 
            else:
                #print ("Third Dominant Freq Chosen as Heart Rate")
                predicted_freqs.append (freqs_ppg[sorted_spectrum_ppg[1][i]])   

    predicted_freqs_array = np.array(predicted_freqs)*60
    #print ("Lengths=", len(predicted_freqs_array), len(ground_truth))
    #print ("Pred BPM =", predicted_freqs_array, "Ground Truth =", ground_truth)
    #plt.clf()
    #plt.figure(0)
    #plt.title('Predictions vs Ground Truth')
    #plt.plot (predicted_freqs_array)
    #plt.plot (ground_truth)
    error = np.abs(predicted_freqs_array - ground_truth)
    
    confidence_list = []
    i = 0
    for i in range(len(predicted_freqs)):
        pred = predicted_freqs[i]
    
        # Convert to frequency domain to parition the area
        # surrounding the signal and the rest of the signal
        freqs = np.fft.rfftfreq(len(bandpassed_ppg)*2, 1/fs)
        fft_mag = np.abs(np.fft.rfft(bandpassed_ppg, len(bandpassed_ppg)*2))

        # Definition of surrounding = Width of the window size
        window_slice = 0.20
        fwindow_size = (freqs > pred - window_slice) & (freqs < pred + window_slice)
        
        # Signal
        sig = np.sum(fft_mag[(fwindow_size)])
        
        # Noise
        noi = np.sum(fft_mag[~(fwindow_size)])

        snr_ratio = sig/noi 
        confidence_list.append(snr_ratio)

    confidence = np.array(confidence_list)

    return error, confidence
